In [1]:
import pandas as pd
import numpy as np
import pickle

import keras
import keras.utils
import spacy

from keras.callbacks import TensorBoard
from keras import optimizers

from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
from keras.utils import pad_sequences
from keras_preprocessing import sequence

In [8]:
# Load data from csv to pandas dataframe
# train_data = pd.read_csv('processed_train_data.tsv', sep='\t', index_col=0)
# val_data = pd.read_csv('processed_val_data.tsv', sep='\t', index_col=0)
# test_data = pd.read_csv('processed_test_data.tsv', sep='\t', index_col=0)

In [54]:
# # Alternative way to load data - pickle to pandas dataframe
# train_data = pd.read_pickle('processed_train_data.p')
# val_data = pd.read_pickle('processed_val_data.p')
# test_data = pd.read_pickle('processed_test_data.p')

In [55]:
# train_data.head()

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy,dep_id,dep_id_custom
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0,2,"[2, 15, 9, 9, 3, 1, 2, 3, 11, 1, 1, 5, 1, 11]","[17, 6, 12, 12, 1, 8, 17, 1, 13, 8, 8, 2, 8, 13]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]","[7, 5, 11, 11, 8, 6, 11, 8, 1, 4, 9, 2, 3, 1]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,1,2,"[8, 2, 15, 1, 5, 1, 1, 11, 6, 2, 8, 3, 1, 2, 5...","[15, 17, 6, 8, 2, 8, 8, 13, 11, 17, 15, 1, 8, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1...","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...","[11, 7, 5, 6, 2, 4, 3, 1, 6, 7, 11, 8, 6, 11, ..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,1,8,"[9, 9, 2, 5, 9, 9, 11, 5, 2, 13, 2, 9, 9, 15, ...","[12, 12, 17, 2, 12, 12, 13, 2, 17, 10, 17, 12,...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274...","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...","[4, 6, 7, 2, 4, 3, 1, 2, 11, 10, 11, 4, 11, 5,..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,2,0,"[1, 1, 1, 1, 12, 3, 13, 2, 3, 1, 1, 1, 11]","[8, 8, 8, 8, 4, 1, 10, 17, 1, 8, 8, 8, 13]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]","[4, 4, 4, 6, 7, 11, 10, 11, 8, 4, 4, 9, 1]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,1,1,"[15, 3, 1, 2, 5, 15, 1, 5, 6, 1, 11]","[6, 1, 8, 17, 2, 6, 8, 2, 11, 8, 13]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]","[5, 8, 6, 7, 2, 5, 3, 2, 11, 3, 1]"


In [2]:
## LOAD SAVED DATA
# load data
train_data = pd.read_pickle('processed_train_data.p')
val_data = pd.read_pickle('processed_val_data.p')
test_data = pd.read_pickle('processed_test_data.p')

# load vocab dicts
vocabulary_dict_custom = pickle.load(open('vocabulary_statement_custom.p', 'rb'))
vocabulary_dict_spacy = pickle.load(open('vocabulary_statement_spacy.p', 'rb'))
vocab_length = len(vocabulary_dict_custom)
print("Length of the vocabulary dictionary:", vocab_length)

# load embeddings matrixes
embedding_matrix_custom_100d = np.load('embedding_matrix_custom_100d.npy')
embedding_matrix_spacy_100d = np.load('embedding_matrix_spacy_100d.npy')
embedding_matrix_custom_300d = np.load('embedding_matrix_custom_300d.npy')
embedding_matrix_spacy_300d = np.load('embedding_matrix_spacy_300d.npy')
print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape, embedding_matrix_spacy_300d.shape)

Length of the vocabulary dictionary: 9606
Shape of the embedding matrixes: (9607, 100) (9496, 300)


In [3]:
# Load DEP_dict
with open('dep_dict.p', 'rb') as f:
    dep_dict, dep_dict_custom = pickle.load(f)

In [4]:
# Load POS dictionaries
with open('pos_dicts.pickle', 'rb') as f:
    pos_dict_custom, pos_dict_default = pickle.load(f)

In [5]:
train_data

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy,dep_id,dep_id_custom
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0,2,"[2, 15, 9, 9, 3, 1, 2, 3, 11, 1, 1, 5, 1, 11]","[17, 6, 12, 12, 1, 8, 17, 1, 13, 8, 8, 2, 8, 13]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]","[7, 5, 11, 11, 8, 6, 11, 8, 1, 4, 9, 2, 3, 1]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,1,2,"[8, 2, 15, 1, 5, 1, 1, 11, 6, 2, 8, 3, 1, 2, 5...","[15, 17, 6, 8, 2, 8, 8, 13, 11, 17, 15, 1, 8, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1...","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...","[11, 7, 5, 6, 2, 4, 3, 1, 6, 7, 11, 8, 6, 11, ..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,1,8,"[9, 9, 2, 5, 9, 9, 11, 5, 2, 13, 2, 9, 9, 15, ...","[12, 12, 17, 2, 12, 12, 13, 2, 17, 10, 17, 12,...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274...","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...","[4, 6, 7, 2, 4, 3, 1, 2, 11, 10, 11, 4, 11, 5,..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,2,0,"[1, 1, 1, 1, 12, 3, 13, 2, 3, 1, 1, 1, 11]","[8, 8, 8, 8, 4, 1, 10, 17, 1, 8, 8, 8, 13]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]","[4, 4, 4, 6, 7, 11, 10, 11, 8, 4, 4, 9, 1]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,1,1,"[15, 3, 1, 2, 5, 15, 1, 5, 6, 1, 11]","[6, 1, 8, 17, 2, 6, 8, 2, 11, 8, 13]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]","[5, 8, 6, 7, 2, 5, 3, 2, 11, 3, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,,Florida,none,0.0,...,2,1,"[6, 2, 15, 3, 1, 5, 1, 1, 5, 9, 8, 6, 2, 1, 5,...","[11, 17, 6, 1, 8, 2, 8, 8, 2, 12, 15, 11, 17, ...",be large number shark attack florida than be c...,large number shark attack florida case voter f...,"[1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...","[126, 100, 5078, 257, 59, 344, 168, 482]","[10, 6, 4, 7, 10, 1, 3, 2, 1, 2, 10, 10, 10, 1...","[11, 7, 5, 8, 11, 2, 4, 3, 2, 3, 11, 11, 11, 1..."
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,,Georgia,republican,0.0,...,0,0,"[9, 12, 4, 2, 15, 1, 5, 15, 15, 9, 11, 9, 1, 1...","[12, 4, 3, 17, 6, 8, 2, 6, 14, 12, 13, 12, 8, ...",democrats have now become party atlanta metro ...,democrats party atlanta metro area black,"[157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]","[122, 169, 397, 1298, 574, 325]","[5, 9, 10, 6, 4, 10, 1, 4, 10, 10, 0, 3, 2, 10...","[6, 10, 11, 7, 5, 11, 2, 5, 11, 11, 1, 4, 3, 1..."
10237,10237,3959.json,half-true,Says an altern

## to test the different preprocessing methods, the following experiments will be run:
1. siddarth - baseline

        - ogdict
        - nltk stopword removal only

****
****

## todo
****
### INITIALISE INPUT/OUTPUT
1. [x] word_id custom AS X_train_custom
  - [x] embedding_matrix_custom
  - [x] vocabulary_dict_custom
2. [x] word_id spacy AS X_train_spacy
  - [x] embedding_matrix_spacy
  - [x] vocabulary_dict_spacy
  
3. [x] pos_id custom AS X_train_pos_custom
4. [x] pos_id spacy AS X_train_pos_spacy
    
#### Input variables to be processed: 
- [x] meta
- [ ] dep parse

****
### GENERAL
- [x] change everything to python 3

### variables, init, etc.
- [x] pass in vocabulary.p
****
### BILSTM MODEL
- [x] functions: train(), etc
        (CODE CELLS COULD BE BETTER, INVESTIGATE)
- [ ] verify varibales are correct

****
*decide which word id to use*
- [x] jaccard similarity

1. fathan - pos tag check [ ]
    - custom pos tag dict
    - spacy preprocess - spacy word id
    - glove
 2. fathan - preprocess [ ]
    - custom pos tag dict
    - custom preprocess - custom word id
    - glove
    * check best results with default pos tag [ ]
****

# HYPERPARAMS

In [59]:
# num_state = len(train_data['state_id'].unique())
# print(num_state)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=num_state)

18


IndexError: index 20 is out of bounds for axis 1 with size 18

In [7]:
# 100d GLOVE
EMBED_DIM = 100

# vocab_length = len(vocabulary_dict.keys())
custom_vocabLen = len(vocabulary_dict_custom.keys())
spacy_vocabLen = len(vocabulary_dict_spacy.keys())
hidden_size = EMBED_DIM #Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 30
num_epochs = 30
batch_size = 40

# embedding_matrix = embedding_matrix_custom_100d

#Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data['state_id'].unique())
num_context = len(train_data['context_id'].unique())
num_job = len(train_data['job_id'].unique())
num_sub = len(train_data['subject_id'].unique())
num_speaker = len(train_data['speaker_id'].unique())

print(num_party)
print(num_state)
print(num_context)
print(num_job)
print(num_sub)
print(num_speaker)
print(train_data.columns)

9
18
18
19
25
26
Index(['index', 'id', 'label', 'statement', 'subject', 'speaker', 'job_title',
       'state_info', 'party', 'barely true', 'false', 'half-true',
       'mostly-true', 'pants-on-fire', 'context', 'output', 'subject_id',
       'speaker_id', 'job_id', 'state_id', 'party_id', 'context_id', 'pos_id',
       'pos_id_DEFAULT', 'statement_custom', 'statement_spacy',
       'word_id_custom', 'word_id_spacy', 'dep_id', 'dep_id_custom'],
      dtype='object')


In [7]:
print(train_data['word_id_custom'])

0        [3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...
1        [37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...
2        [104, 69, 734, 160, 201, 18, 89, 262, 137, 258...
3        [19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...
4                               [282, 3331, 308, 247, 248]
                               ...                        
10235    [1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...
10236       [157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]
10237    [3, 2024, 154, 110, 1205, 3983, 124, 38, 2, 42...
10238       [1604, 28, 13, 2857, 3115, 4, 153, 1391, 1473]
10239    [216, 1429, 2087, 2, 5142, 826, 494, 327, 441,...
Name: word_id_custom, Length: 10240, dtype: object


In [8]:
print(train_data['dep_id'])

0            [6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]
1        [10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...
2        [3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...
3                [3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]
4                       [4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]
                               ...                        
10235    [10, 6, 4, 7, 10, 1, 3, 2, 1, 2, 10, 10, 10, 1...
10236    [5, 9, 10, 6, 4, 10, 1, 4, 10, 10, 0, 3, 2, 10...
10237    [6, 4, 5, 1, 3, 2, 5, 10, 1, 3, 2, 0, 10, 0, 9...
10238             [6, 10, 4, 10, 7, 8, 10, 10, 8, 1, 2, 0]
10239    [4, 5, 1, 3, 2, 6, 4, 8, 10, 10, 10, 10, 10, 8...
Name: dep_id, Length: 10240, dtype: object


In [11]:
print(train_data['party_id'])

0        0
1        1
2        1
3        2
4        1
        ..
10235    2
10236    0
10237    0
10238    1
10239    0
Name: party_id, Length: 10240, dtype: int64


# Input/Output Processing

In [63]:
# party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=11)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=51)
# context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=21)

In [57]:
# party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=11)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=21)
# context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=21)

## Apr 6 TODO:
    - [ ] fix the input shape for the meta data, as num_party is 11, not 9.
            - num_party = len(train_data.party_id.unique()) returns 9 instead of 11.
            - or is it expecting 11 wrongly and party_id number of labels should be

In [8]:

#Meta data preparation
party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=num_party+2)
state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=num_state+3)
context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=num_context+3)
job_train = keras.utils.to_categorical(train_data['job_id'], num_classes=num_job+3)
subject_train = keras.utils.to_categorical(train_data['subject_id'], num_classes=num_sub+3)
speaker_train = keras.utils.to_categorical(train_data['speaker_id'], num_classes=num_speaker+2)


party_val = keras.utils.to_categorical(val_data['party_id'], num_classes=num_party+2)
state_val = keras.utils.to_categorical(val_data['state_id'], num_classes=num_state+3)
context_val = keras.utils.to_categorical(val_data['context_id'], num_classes=num_context+3)
job_val = keras.utils.to_categorical(val_data['job_id'], num_classes=num_job+3)
subject_val = keras.utils.to_categorical(val_data['subject_id'], num_classes=num_sub+3)
speaker_val = keras.utils.to_categorical(val_data['speaker_id'], num_classes=num_speaker+2)


party_test = keras.utils.to_categorical(test_data['party_id'], num_classes=num_party+2)
state_test = keras.utils.to_categorical(test_data['state_id'], num_classes=num_state+3)
context_test = keras.utils.to_categorical(test_data['context_id'], num_classes=num_context+3)
job_test = keras.utils.to_categorical(test_data['job_id'], num_classes=num_job+3)
subject_test = keras.utils.to_categorical(test_data['subject_id'], num_classes=num_sub+3)
speaker_test = keras.utils.to_categorical(test_data['speaker_id'], num_classes=num_speaker+2)

In [9]:
def input_pad_sequences(data):
    padded_data = sequence.pad_sequences(data, maxlen=num_steps, padding='post', truncating='post')
    return padded_data

X_train_custom = input_pad_sequences(train_data['word_id_custom'])
X_val_custom = input_pad_sequences(val_data['word_id_custom'])
X_test_custom = input_pad_sequences(test_data['word_id_custom'])

X_train_spacy = input_pad_sequences(train_data['word_id_spacy'])
X_val_spacy = input_pad_sequences(val_data['word_id_spacy'])
X_test_spacy = input_pad_sequences(test_data['word_id_spacy'])

X_train_pos_custom = input_pad_sequences(train_data['pos_id'])
X_val_pos_custom = input_pad_sequences(val_data['pos_id'])
X_test_pos_custom = input_pad_sequences(test_data['pos_id'])

X_train_pos_DEFAULT = input_pad_sequences(train_data['pos_id_DEFAULT'])
X_val_pos_DEFAULT = input_pad_sequences(val_data['pos_id_DEFAULT'])
X_test_pos_DEFAULT = input_pad_sequences(test_data['pos_id_DEFAULT'])

X_train_meta = np.hstack((party_train,
                          state_train,
                          context_train,
                          job_train,
                          subject_train,
                          speaker_train))

X_val_meta = np.hstack((party_val,
                        state_val,
                        context_val,
                        job_val,
                        subject_val,
                        speaker_val))

X_test_meta = np.hstack((party_test,
                         state_test,
                         context_test,
                         job_test,
                         subject_test,
                         speaker_test))

X_train_dep = input_pad_sequences(train_data['dep_id_custom'])
X_val_dep = input_pad_sequences(val_data['dep_id_custom'])
X_test_dep = input_pad_sequences(test_data['dep_id_custom'])

In [11]:
train_data.head()

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy,dep_id,dep_id_custom
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0,2,"[2, 15, 9, 9, 3, 1, 2, 3, 11, 1, 1, 5, 1, 11]","[17, 6, 12, 12, 1, 8, 17, 1, 13, 8, 8, 2, 8, 13]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]","[7, 5, 11, 11, 8, 6, 11, 8, 1, 4, 9, 2, 3, 1]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,1,2,"[8, 2, 15, 1, 5, 1, 1, 11, 6, 2, 8, 3, 1, 2, 5...","[15, 17, 6, 8, 2, 8, 8, 13, 11, 17, 15, 1, 8, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1...","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...","[11, 7, 5, 6, 2, 4, 3, 1, 6, 7, 11, 8, 6, 11, ..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,1,8,"[9, 9, 2, 5, 9, 9, 11, 5, 2, 13, 2, 9, 9, 15, ...","[12, 12, 17, 2, 12, 12, 13, 2, 17, 10, 17, 12,...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274...","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...","[4, 6, 7, 2, 4, 3, 1, 2, 11, 10, 11, 4, 11, 5,..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,2,0,"[1, 1, 1, 1, 12, 3, 13, 2, 3, 1, 1, 1, 11]","[8, 8, 8, 8, 4, 1, 10, 17, 1, 8, 8, 8, 13]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]","[4, 4, 4, 6, 7, 11, 10, 11, 8, 4, 4, 9, 1]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,1,1,"[15, 3, 1, 2, 5, 15, 1, 5, 6, 1, 11]","[6, 1, 8, 17, 2, 6, 8, 2, 11, 8, 13]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]","[5, 8, 6, 7, 2, 5, 3, 2, 11, 3, 1]"


In [26]:
print(X_train_dep.shape)

(10240, 30)


In [30]:
print(X_train_pos_custom)

[[ 2 15  9 ...  0  0  0]
 [ 8  2 15 ... 11  0  0]
 [ 9  9  2 ...  0  0  0]
 ...
 [ 2 15  1 ...  5  9  9]
 [ 5  2 15 ...  0  0  0]
 [15  9  5 ...  6  2 11]]


In [32]:
print(X_train_meta.shape, X_val_meta.shape, X_test_meta.shape)

(10240, 131) (1284, 131) (1267, 131)


In [12]:
# Initialise input/output

## OUTPUT
Y_train = train_data['output']
Y_train = keras.utils.to_categorical(Y_train, num_classes=6)

Y_val = val_data['output']
Y_val = keras.utils.to_categorical(Y_val, num_classes=6)

In [34]:
print(X_test_custom)
print(X_test_spacy)
print(X_test_pos_custom)
print(X_test_pos_DEFAULT)
print(X_train_meta)

[[ 350  425   28 ...    0    0    0]
 [  64    1 1930 ...    0    0    0]
 [   3  160  201 ...    0    0    0]
 ...
 [ 472 1547  191 ...    0    0    0]
 [   3 1406  928 ...    0    0    0]
 [   3   93    1 ...    0    0    0]]
[[ 291  358   19 ...    0    0    0]
 [  45 1829  234 ...    0    0    0]
 [   1  125  157 ...    0    0    0]
 ...
 [ 405 1448  153 ...    0    0    0]
 [   1 1306  836 ...    0    0    0]
 [   1   64   34 ...    0    0    0]]
[[ 2 15  1 ...  0  0  0]
 [ 9 12  5 ...  0  0  0]
 [ 2  9  9 ...  0  0  0]
 ...
 [ 5 15  3 ...  0  0  0]
 [ 2 15  9 ...  0  0  0]
 [ 2 15  1 ...  3  1  1]]
[[17  6  8 ...  0  0  0]
 [12  4  2 ...  0  0  0]
 [17 12 12 ...  0  0  0]
 ...
 [ 2  6  1 ...  0  0  0]
 [17  6 12 ...  0  0  0]
 [17  6  8 ...  1  8  8]]
[[1. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [27]:
print(X_train_custom.shape, X_val_custom.shape, X_test_custom.shape)
print(X_train_spacy.shape, X_val_spacy.shape, X_test_spacy.shape)
print(X_train_pos_custom.shape, X_val_pos_custom.shape, X_test_pos_custom.shape)
print(X_train_pos_DEFAULT.shape, X_val_pos_DEFAULT.shape, X_test_pos_DEFAULT.shape)
print(Y_train.shape, Y_val.shape)

(10240, 30) (1284, 30) (1267, 30)
(10240, 30) (1284, 30) (1267, 30)
(10240, 30) (1284, 30) (1267, 30)
(10240, 30) (1284, 30) (1267, 30)
(10240, 6) (1284, 6)


#### 1. fathan - pos tag check [ ]
    - custom word id [ ]
    - custom pos id [ ]
    - glove [ ]

In [27]:
max_value = max(np.max(X_train_pos_custom), np.max(X_val_pos_custom))
print("Maximum value in the input data:", max_value)

Maximum value in the input data: 15


In [10]:
import datetime

def train(model, name, train_inputs, val_inputs, optimizer, output_name):
    """
    Trains the given model with the provided input tensors.

    :param model: {tf.keras.Model} model to be trained
    :param name: {str} model name used for saving best weights
    :param train_inputs: {dict} dictionary containing training input tensors
    :param val_inputs: {dict} dictionary containing validation input tensors
    :param optimizer: {keras.optimizers} optimizer to be used
    :param output_name: {str} main output name used in the model
    """

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    tb = TensorBoard()
    csv_logger = keras.callbacks.CSVLogger('training.log')
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = f"{name}_weights_best_{timestamp}.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                                  verbose=1, save_best_only=True, mode='max', save_freq='epoch')

    model.fit(
        train_inputs,
        {output_name: Y_train},
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(val_inputs, {output_name: Y_val}),
        callbacks=[tb, csv_logger, checkpoint]
    )

In [11]:
import operator

def print_best_false_true_predicted(fw, tb):
  sorted_false = sorted(fw.items(), key=operator.itemgetter(1), reverse=True)
  sorted_true = sorted(tb.items(), key=operator.itemgetter(1), reverse=True)
  print('*****************************************************************')
  print('******************** False statements *************************')

  for t in sorted_false[:5]:
    print(t[1])
    print(test_data.loc[t[0]])
    print('=============')
  print('*****************************************************************')
  print('******************** True Statements *************************')
  for t in sorted_true[:5]:
    print(t[1])
    print(test_data.loc[t[0]])
    print('=============')


In [12]:


def calculate_accuracy(predictions, Y_test_gt):
    """
    Calculate the accuracy of the predictions

    :param predictions: {np.array} predicted values
    :param Y_test_gt: {np.array} ground truth values

    :return: accuracy {float} accuracy of the predictions
    """
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == Y_test_gt[i]:
            correct += 1
    accuracy = correct / len(predictions)
    return accuracy

def extract_fw_tb(preds):
    """
    Extracts the worst false predictions and the best true predictions

    :param preds: {np.array} array of predicted values

    :return: false_worst: {dict} dictionary of worst false predictions
    :return: true_best: {dict} dictionary of best true predictions
    """
    false_worst = {}
    true_best = {}

    for p in range(len(preds)):
        if np.argmax(preds[p])==0:
            false_worst[p]=preds[p][0]
        elif np.argmax(preds[p])==5:
            true_best[p]=preds[p][5]

    return false_worst, true_best

In [13]:
def load_and_predict(model_filename, inputs):
    """
    Load the model and predict with the provided inputs

    :param name: {str} name of the model to load the model weights
    :param inputs: {dict} dictionary of inputs to be passed to the model

    :return: preds {np.array} array of predicted values/
    """
    model = load_model(model_filename)
    preds = model.predict(inputs,
                          batch_size=batch_size,
                          verbose=1)
    return preds

In [14]:
def evaluate(model_name, *args, **kwargs):
    """
    Evaluates the given model with provided input tensors

    :param model_name: {str} used to load model weights
    :param args: {list} of dictionaries of inputs
    :param kwargs:

    :return: false_worst: {dict} of the worst false predictions
    :return: true_best: {dict} of the best true predictions
    """
    inputs = {}
    for arg in args:
        inputs.update(arg)
    inputs.update(kwargs)

    preds = load_and_predict(model_name, inputs)

    Y_test_groundtruth = list(test_data['output'])
    predictions = np.array([np.argmax(pred) for pred in preds])

    accuracy = calculate_accuracy(predictions, Y_test_groundtruth)
    print("Correctly Predicted: ", np.sum(predictions == Y_test_groundtruth), "/", len(Y_test_groundtruth))
    print("Accuracy: ", accuracy)

    false_worst, true_best = extract_fw_tb(preds)

    pickle.dump(predictions, open(model_name + "_predictions.p", "wb"))
    return false_worst, true_best

##

In [122]:
# model_name = "bilstm"
#
# for config in configurations:
#     print(f"Running {config['name']}...")
#     (fw, tb) = evaluate(model_name, *config["inputs"])


In [123]:
train_data

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy,dep_id,dep_id_custom
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0,2,"[2, 15, 9, 9, 3, 1, 2, 3, 11, 1, 1, 5, 1, 11]","[17, 6, 12, 12, 1, 8, 17, 1, 13, 8, 8, 2, 8, 13]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]","[7, 5, 11, 11, 8, 6, 11, 8, 1, 4, 9, 2, 3, 1]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,1,2,"[8, 2, 15, 1, 5, 1, 1, 11, 6, 2, 8, 3, 1, 2, 5...","[15, 17, 6, 8, 2, 8, 8, 13, 11, 17, 15, 1, 8, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1...","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...","[11, 7, 5, 6, 2, 4, 3, 1, 6, 7, 11, 8, 6, 11, ..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,1,8,"[9, 9, 2, 5, 9, 9, 11, 5, 2, 13, 2, 9, 9, 15, ...","[12, 12, 17, 2, 12, 12, 13, 2, 17, 10, 17, 12,...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274...","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...","[4, 6, 7, 2, 4, 3, 1, 2, 11, 10, 11, 4, 11, 5,..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,2,0,"[1, 1, 1, 1, 12, 3, 13, 2, 3, 1, 1, 1, 11]","[8, 8, 8, 8, 4, 1, 10, 17, 1, 8, 8, 8, 13]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]","[4, 4, 4, 6, 7, 11, 10, 11, 8, 4, 4, 9, 1]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,1,1,"[15, 3, 1, 2, 5, 15, 1, 5, 6, 1, 11]","[6, 1, 8, 17, 2, 6, 8, 2, 11, 8, 13]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]","[5, 8, 6, 7, 2, 5, 3, 2, 11, 3, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,,Florida,none,0.0,...,2,1,"[6, 2, 15, 3, 1, 5, 1, 1, 5, 9, 8, 6, 2, 1, 5,...","[11, 17, 6, 1, 8, 2, 8, 8, 2, 12, 15, 11, 17, ...",be large number shark attack florida than be c...,large number shark attack florida case voter f...,"[1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...","[126, 100, 5078, 257, 59, 344, 168, 482]","[10, 6, 4, 7, 10, 1, 3, 2, 1, 2, 10, 10, 10, 1...","[11, 7, 5, 8, 11, 2, 4, 3, 2, 3, 11, 11, 11, 1..."
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,,Georgia,republican,0.0,...,0,0,"[9, 12, 4, 2, 15, 1, 5, 15, 15, 9, 11, 9, 1, 1...","[12, 4, 3, 17, 6, 8, 2, 6, 14, 12, 13, 12, 8, ...",democrats have now become party atlanta metro ...,democrats party atlanta metro area black,"[157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]","[122, 169, 397, 1298, 574, 325]","[5, 9, 10, 6, 4, 10, 1, 4, 10, 10, 0, 3, 2, 10...","[6, 10, 11, 7, 5, 11, 2, 5, 11, 11, 1, 4, 3, 1..."
10237,10237,3959.json,half-true,Says an altern

In [17]:
# print(len(embedding_matrix))
print(len(embedding_matrix_spacy_100d))

9496


In [18]:
# embedding_matrix_glove_wordcustom = embedding_matrix_custom_100d
# embedding_matrix_glove_wordspacy = embedding_matrix_spacy_100d

# statement embed bilstm
vocab_length_custom = len(vocabulary_dict_custom.keys())
statement_input_custom = Input(shape=num_steps, dtype='int32', name='main_input_custom')
x = Embedding(vocab_length_custom+1,EMBED_DIM,weights=[embedding_matrix_custom_100d],input_length=num_steps,trainable=False)(statement_input_custom)
bilstm_word_in_custom = LSTM(lstm_size, dropout=0.2, name='bilstm_word_in_custom')(x)

vocab_length_spacy = len(vocabulary_dict_spacy.keys())
statement_input_spacy = Input(shape=num_steps, dtype='int32', name='main_input_spacy')
x_spacy = Embedding(vocab_length_spacy+1,EMBED_DIM,weights=[embedding_matrix_spacy_100d],input_length=num_steps,trainable=False)(statement_input_spacy)
bilstm_word_in_spacy = LSTM(lstm_size, dropout=0.2, name='bilstm_word_in_spacy')(x_spacy)

# SPECIFY POS_DICT USED FOR POS EMBEDDING
pos_embeddings_custom = np.identity(max(pos_dict_custom.values())+1, dtype=int)
pos_input_custom = Input(shape=num_steps, dtype='int32', name='pos_input_custom')
xpos_custom = Embedding(max(pos_dict_custom.values())+1, max(pos_dict_custom.values())+1, weights=[pos_embeddings_custom], input_length=num_steps, trainable=False)(pos_input_custom)
bilstm_pos_in_custom = LSTM(lstm_size, dropout=0.2, name='bilstm_pos_in_custom')(xpos_custom)

pos_embeddings_default = np.identity(max(pos_dict_default.values())+1, dtype=int)
pos_input_default = Input(shape=num_steps, dtype='int32', name='pos_input_default')
xpos_default = Embedding(max(pos_dict_default.values())+1, max(pos_dict_default.values())+1, weights=[pos_embeddings_default], input_length=num_steps, trainable=False)(pos_input_default)
bilstm_pos_in_default = LSTM(lstm_size, dropout=0.2, name='bilstm_pos_in_default')(xpos_default)

# SPECIFY DEP_DICT USED FOR DEP EMBEDDING
dep_embeddings_custom = np.identity(max(dep_dict_custom.values())+1, dtype=int)
# dep embed LSTM X3
dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
xdep = Embedding(max(dep_dict_custom.values())+1, max(dep_dict_custom.values())+1, weights=[dep_embeddings_custom], input_length=num_steps, trainable=False)(dep_input)
bilstm_dep_custom_in = LSTM(lstm_size, dropout=0.2)(xdep)

# meta data Dense layer
meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = Dense(64, activation='relu')(meta_input)

## Train

In [19]:
def concat_lstm_layers(bilstm_input, layers_to_concat):
    layers = [bilstm_input] + layers_to_concat
    x = keras.layers.concatenate(layers)
    return x

In [20]:
layers_poscustom_depmeta = [bilstm_pos_in_custom, bilstm_dep_custom_in, x_meta]
layers_posdefault_depmeta = [bilstm_pos_in_default, bilstm_dep_custom_in, x_meta]

concat_customcustom = concat_lstm_layers(bilstm_word_in_custom, layers_poscustom_depmeta)
main_output_customcustom = Dense(6, activation='softmax', name='main_output_customcustom')(concat_customcustom)
experiment_customcustom_input = [statement_input_custom, pos_input_custom, dep_input, meta_input]

concat_customdefault = concat_lstm_layers(bilstm_word_in_custom, layers_posdefault_depmeta)
main_output_customdefault = Dense(6, activation='softmax', name='main_output_customdefault')(concat_customdefault)
experiment_customdefault_input = [statement_input_custom, pos_input_default, dep_input, meta_input]

concat_spacycustom = concat_lstm_layers(bilstm_word_in_spacy, layers_poscustom_depmeta)
main_output_spacycustom = Dense(6, activation='softmax', name='main_output_spacycustom')(concat_spacycustom)
experiment_spacycustom_input = [statement_input_spacy, pos_input_custom, dep_input, meta_input]

concat_spacydefault = concat_lstm_layers(bilstm_word_in_spacy, layers_posdefault_depmeta)
main_output_spacydefault = Dense(6, activation='softmax', name='main_output_spacydefault')(concat_spacydefault)
experiment_spacydefault_input = [statement_input_spacy, pos_input_default, dep_input, meta_input]

In [21]:
# model_bilstm_customcustom = Sequential()
# model_bilstm_customcustom.add(Embedding(vocab_length_custom+1, hidden_size, input_length=num_steps))
# model_bilstm_customcustom.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_customcustom.add(Dense(6, activation='softmax'))
model_bilstm_customcustom = Model(inputs=experiment_customcustom_input, outputs=main_output_customcustom, name='model_bilstm_customcustom')

# model_bilstm_customdefault = Sequential()
# model_bilstm_customdefault.add(Embedding(vocab_length_custom+1, hidden_size, input_length=num_steps))
# model_bilstm_customdefault.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_customdefault.add(Dense(6, activation='softmax'))
model_bilstm_customdefault = Model(inputs=experiment_customdefault_input, outputs=main_output_customdefault,name='model_bilstm_customdefault')

# model_bilstm_spacycustom = Sequential()
# model_bilstm_spacycustom.add(Embedding(vocab_length_spacy+1, hidden_size, input_length=num_steps))
# model_bilstm_spacycustom.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_spacycustom.add(Dense(6, activation='softmax'))
model_bilstm_spacycustom = Model(inputs=experiment_spacycustom_input, outputs=main_output_spacycustom,name='model_bilstm_spacycustom')

# model_bilstm_spacydefault = Sequential()
# model_bilstm_spacydefault.add(Embedding(vocab_length_spacy+1, hidden_size, input_length=num_steps))
# model_bilstm_spacydefault.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_spacydefault.add(Dense(6, activation='softmax'))
model_bilstm_spacydefault = Model(inputs=experiment_spacydefault_input, outputs=main_output_spacydefault,name='model_bilstm_spacydefault')

In [22]:
model_bilstm_spacydefault.summary()
model_bilstm_spacycustom.summary()

Model: "model_bilstm_spacydefault"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input_spacy (InputLayer)  [(None, 30)]         0           []                               
                                                                                                  
 pos_input_default (InputLayer)  [(None, 30)]        0           []                               
                                                                                                  
 dep_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      949600      ['main_input_spacy[0][0]']       
                                                                          

In [93]:
model_bilstm_spacydefault.summary()
model_bilstm_spacycustom.summary()

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 dep_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_76 (Embedding)       (None, 30, 100)      949600      ['main_input[0][0]']             
                                                                                           

In [36]:
model_bilstm_customcustom.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 dep_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_30 (Embedding)       (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                              

In [51]:
# extra_layers_pos_custom = [bilstm_pos_in_custom]
# extra_layer_pos_default = [bilstm_pos_in_default]
# extra_layers_for_pos_meta = [bilstm_pos_in_custom, x_meta]
# extra_layers_for_poscustom_dep_meta = [bilstm_pos_in_custom, bilstm_dep_custom_in, x_meta]
#
# concat_pos = concat_lstm_layers(bilstm_word_input, extra_layers_pos_custom)
# main_output_pos = Dense(6, activation='softmax', name='main_output')(concat_pos)
# experiment1input = [statement_input, pos_input_custom]
#
# concat_pos_meta = concat_lstm_layers(bilstm_word_input, extra_layers_for_pos_meta)
# main_output_pos_meta = Dense(6, activation='softmax', name='main_output')(concat_pos_meta)
# experiment2input = [statement_input, pos_input_custom, meta_input]
#
# concat_pos_dep_meta = concat_lstm_layers(bilstm_word_input, extra_layers_for_poscustom_dep_meta)
# main_output_dep_meta = Dense(6, activation='softmax', name='main_output')(concat_pos_dep_meta)
# experiment3input = [statement_input, pos_input_custom, dep_input, meta_input]
#
# model_bilstm_pos = Sequential()
# model_bilstm_pos.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
# model_bilstm_pos.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_pos.add(Dense(6, activation='softmax'))
# model_bilstm_pos = Model(inputs=experiment1input, outputs=[main_output_pos])
#
# model_bilstm_meta = Sequential()
# model_bilstm_meta.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
# model_bilstm_meta.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_meta.add(Dense(6, activation='softmax'))
# model_bilstm_meta = Model(inputs=experiment2input, outputs=[main_output_pos_meta])
#
# model_bilstm_dep_meta = Sequential()
# model_bilstm_dep_meta.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
# model_bilstm_dep_meta.add(Bidirectional(LSTM(hidden_size)))
# model_bilstm_dep_meta.add(Dense(6, activation='softmax'))
# model_bilstm_dep_meta = Model(inputs=experiment3input, outputs=[main_output_dep_meta])

In [41]:
model_bilstm_pos.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_2 (Embedding)        (None, 30, 18)       324         ['pos_input[0][0]']              
                                                                                            

In [43]:
# MOst recent 342am
model_bilstm_dep_meta.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 dep_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                           

In [40]:
model_bilstm_dep_meta.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 dep_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                            

In [37]:
model_bilstm_dep_meta.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_2 (Embedding)        (None, 30, 18)       324         ['pos_input[0][0]']              
                                                                                            

In [50]:
model_bilstm_meta.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_2 (Embedding)        (None, 30, 16)       256         ['pos_input[0][0]']              
                                                                                            

In [51]:
model_bilstm.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_2 (Embedding)        (None, 30, 16)       256         ['pos_input[0][0]']              
                                                                                            

In [97]:
model_bilstm.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 embedding_36 (Embedding)       (None, 50, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_37 (Embedding)       (None, 50, 15)       225         ['pos_input[0][0]']              
                                                                                            

In [83]:
model_bilstm.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 embedding_25 (Embedding)       (None, 50, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_26 (Embedding)       (None, 50, 14)       196         ['pos_input[0][0]']              
                                                                                            

In [59]:
model_bilstm.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 20)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 20)]         0           []                               
                                                                                                  
 embedding_22 (Embedding)       (None, 20, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_23 (Embedding)       (None, 20, 14)       196         ['pos_input[0][0]']              
                                                                                            

In [ ]:
vocab_length

### APRIL 8th

In [95]:
# vocab_length = len(vocabulary_dict_custom)

print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape)
print("Length of the vocabulary dictionary:", vocab_length)
print("X_train_custom shape:", X_train_custom.shape)
print("X_val_custom shape:", X_val_custom.shape)

Shape of the embedding matrixes: (9607, 100)
Length of the vocabulary dictionary: 9606
X_train_custom shape: (10240, 50)
X_val_custom shape: (1284, 50)


### Config 1

In [47]:
# vocab_length = len(vocabulary_dict_custom)

print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape)
print("Length of the vocabulary dictionary:", vocab_length)
print("X_train_custom shape:", X_train_custom.shape)
print("X_val_custom shape:", X_val_custom.shape)

Shape of the embedding matrixes: (9607, 100)
Length of the vocabulary dictionary: 9606
X_train_custom shape: (10240, 14)
X_val_custom shape: (1284, 14)


In [85]:
print("Max index in X_train_custom:", np.max(X_train_custom))
print("Max index in X_val_custom:", np.max(X_val_custom))


Max index in X_train_custom: 9606
Max index in X_val_custom: 9545


In [27]:
word_index = {word: i+1 for i, word in enumerate(vocabulary_dict_custom)}

In [29]:
def decode_sequence(sequence, index_to_word):
    return " ".join([index_to_word.get(idx, "") for idx in sequence])

index_to_word = {i: word for word, i in word_index.items()}

# Print a decoded sample from X_train_custom
sample_idx = 89  # You can change this to any valid index
print(decode_sequence(X_train_custom[sample_idx], index_to_word))

# Print a decoded sample from X_val_custom
sample_idx = 2  # You can change this to any valid index
print(decode_sequence(X_val_custom[sample_idx], index_to_word))


austin be burden fast grow tax increase major city nation     
say have organization parade be social welfare organization and then be involve political combat back


In [1]:
train_data.head()

NameError: name 'train_data' is not defined

In [8]:
train_data

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy,dep_id,dep_id_custom
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0,2,"[2, 15, 9, 9, 3, 1, 2, 3, 11, 1, 1, 5, 1, 11]","[17, 6, 12, 12, 1, 8, 17, 1, 13, 8, 8, 2, 8, 13]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]","[7, 5, 11, 11, 8, 6, 11, 8, 1, 4, 9, 2, 3, 1]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,1,2,"[8, 2, 15, 1, 5, 1, 1, 11, 6, 2, 8, 3, 1, 2, 5...","[15, 17, 6, 8, 2, 8, 8, 13, 11, 17, 15, 1, 8, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1...","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, ...","[11, 7, 5, 6, 2, 4, 3, 1, 6, 7, 11, 8, 6, 11, ..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,1,8,"[9, 9, 2, 5, 9, 9, 11, 5, 2, 13, 2, 9, 9, 15, ...","[12, 12, 17, 2, 12, 12, 13, 2, 17, 10, 17, 12,...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274...","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, ...","[4, 6, 7, 2, 4, 3, 1, 2, 11, 10, 11, 4, 11, 5,..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,2,0,"[1, 1, 1, 1, 12, 3, 13, 2, 3, 1, 1, 1, 11]","[8, 8, 8, 8, 4, 1, 10, 17, 1, 8, 8, 8, 13]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]","[4, 4, 4, 6, 7, 11, 10, 11, 8, 4, 4, 9, 1]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,1,1,"[15, 3, 1, 2, 5, 15, 1, 5, 6, 1, 11]","[6, 1, 8, 17, 2, 6, 8, 2, 11, 8, 13]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]","[5, 8, 6, 7, 2, 5, 3, 2, 11, 3, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,,Florida,none,0.0,...,2,1,"[6, 2, 15, 3, 1, 5, 1, 1, 5, 9, 8, 6, 2, 1, 5,...","[11, 17, 6, 1, 8, 2, 8, 8, 2, 12, 15, 11, 17, ...",be large number shark attack florida than be c...,large number shark attack florida case voter f...,"[1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...","[126, 100, 5078, 257, 59, 344, 168, 482]","[10, 6, 4, 7, 10, 1, 3, 2, 1, 2, 10, 10, 10, 1...","[11, 7, 5, 8, 11, 2, 4, 3, 2, 3, 11, 11, 11, 1..."
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,,Georgia,republican,0.0,...,0,0,"[9, 12, 4, 2, 15, 1, 5, 15, 15, 9, 11, 9, 1, 1...","[12, 4, 3, 17, 6, 8, 2, 6, 14, 12, 13, 12, 8, ...",democrats have now become party atlanta metro ...,democrats party atlanta metro area black,"[157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]","[122, 169, 397, 1298, 574, 325]","[5, 9, 10, 6, 4, 10, 1, 4, 10, 10, 0, 3, 2, 10...","[6, 10, 11, 7, 5, 11, 2, 5, 11, 11, 1, 4, 3, 1..."
10237,10237,3959.json,half-true,Says an altern

In [23]:
sgd = optimizers.SGD(learning_rate=0.025, clipvalue=0.3, nesterov=True)
adam = optimizers.Adam(learning_rate=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

### APRIL 13th

In [24]:
# word_id_spacy + pos_id_default
train(model_bilstm_spacydefault,
      'spacydefault-glove-adam',
      {'main_input_spacy': X_train_spacy, 'pos_input_default': X_train_pos_DEFAULT, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_spacy': X_val_spacy, 'pos_input_default': X_val_pos_DEFAULT, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam,
      output_name='main_output_spacydefault'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.6671 - categorical_accuracy: 0.2292
Epoch 1: val_categorical_accuracy improved from -inf to 0.23832, saving model to spacydefault-glove-adam_weights_best_20230413_005106.hdf5
256/256 [==============================] - 15s 31ms/step - loss: 1.6671 - categorical_accuracy: 0.2292 - val_loss: 1.6109 - val_categorical_accuracy: 0.2383
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.5966 - categorical_accuracy: 0.2521
Epoch 2: val_categorical_accuracy did not improve from 0.23832
256/256 [==============================] - 7s 26ms/step - loss: 1.5966 - categorical_accuracy: 0.2522 - val_loss: 1.6019 - val_categorical_accuracy: 0.2383
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5887 - categorical_accuracy: 0.2575
Epoch 3: val_categorical_accuracy improved from 0.23832 to 0.23910, saving model to spacydefault-glove-adam_weights_best_20230413_005106.hdf5
256/256 [==========

#### Evalulation

In [19]:
model_file = 'spacydefault-glove-adam_weights_best_20230413_005106.hdf5'
# word_id_spacy + pos_id_default
(fw, tb) = evaluate(model_file,
                    {'main_input_spacy': X_test_spacy,
                     'pos_input_default': X_test_pos_DEFAULT,
                     'dep_input': X_test_dep,
                     'aux_input': X_test_meta}
                    )

print_best_false_true_predicted(fw, tb)

32/32 [==============================] - 2s 11ms/step
Correctly Predicted:  207 / 1267
Accuracy:  0.1633780584056827
*****************************************************************
******************** False statements *************************
*****************************************************************
******************** True Statements *************************
0.362695
index                                                             602
id                                                          4579.json
label                                                           false
statement           Says Bill Clinton opposes President Barack Oba...
subject                                         message-machine,taxes
speaker                                           american-crossroads
job_title                                                            
state_info                                                           
party                                                  

### word_id_custom + pos_id_default

In [25]:
# word_id_custom + pos_id_default
train(model_bilstm_customdefault,
      'customdefault-glove-adam',
      {'main_input_custom': X_train_custom, 'pos_input_default': X_train_pos_DEFAULT, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_custom': X_val_custom, 'pos_input_default': X_val_pos_DEFAULT, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam,
      output_name='main_output_customdefault'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.6191 - categorical_accuracy: 0.2329
Epoch 1: val_categorical_accuracy improved from -inf to 0.22741, saving model to customdefault-glove-adam_weights_best_20230413_005815.hdf5
256/256 [==============================] - 15s 32ms/step - loss: 1.6191 - categorical_accuracy: 0.2329 - val_loss: 1.6102 - val_categorical_accuracy: 0.2274
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5901 - categorical_accuracy: 0.2523
Epoch 2: val_categorical_accuracy improved from 0.22741 to 0.25779, saving model to customdefault-glove-adam_weights_best_20230413_005815.hdf5
256/256 [==============================] - 7s 27ms/step - loss: 1.5901 - categorical_accuracy: 0.2523 - val_loss: 1.5970 - val_categorical_accuracy: 0.2578
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5802 - categorical_accuracy: 0.2613
Epoch 3: val_categorical_accuracy did not improve from 0.25779
256/256 [========

#### Evalulation

In [18]:
model_file = 'customdefault-glove-adam_weights_best_20230413_005815.hdf5'
# word_id_spacy + pos_id_default
(fw, tb) = evaluate(model_file,
                    {'main_input_custom': X_test_custom,
                     'pos_input_default': X_test_pos_DEFAULT,
                     'dep_input': X_test_dep,
                     'aux_input': X_test_meta}
                    )

print_best_false_true_predicted(fw, tb)

32/32 [==============================] - 2s 12ms/step
Correctly Predicted:  195 / 1267
Accuracy:  0.15390686661404893
*****************************************************************
******************** False statements *************************
*****************************************************************
******************** True Statements *************************
0.4116359
index                                                            1216
id                                                          2533.json
label                                                       half-true
statement           Newspapers say Florida made bad investments, l...
subject             financial-regulation,message-machine,retiremen...
speaker                                      republican-party-florida
job_title                                                            
state_info                                                           
party                                                

### word_id_spacy + pos_id_custom

In [26]:
# word_id_spacy + pos_id_custom
train(model_bilstm_spacycustom,
      'spacycustom-glove-adam',
      {'main_input_spacy': X_train_spacy, 'pos_input_custom': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_spacy': X_val_spacy, 'pos_input_custom': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam,
      output_name='main_output_spacycustom'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.5976 - categorical_accuracy: 0.2659
Epoch 1: val_categorical_accuracy improved from -inf to 0.26090, saving model to spacycustom-glove-adam_weights_best_20230413_010320.hdf5
256/256 [==============================] - 14s 31ms/step - loss: 1.5976 - categorical_accuracy: 0.2659 - val_loss: 1.5914 - val_categorical_accuracy: 0.2609
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.5425 - categorical_accuracy: 0.2994
Epoch 2: val_categorical_accuracy improved from 0.26090 to 0.27414, saving model to spacycustom-glove-adam_weights_best_20230413_010320.hdf5
256/256 [==============================] - 7s 26ms/step - loss: 1.5424 - categorical_accuracy: 0.2993 - val_loss: 1.5807 - val_categorical_accuracy: 0.2741
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5335 - categorical_accuracy: 0.3114
Epoch 3: val_categorical_accuracy did not improve from 0.27414
256/256 [============

#### Evalulation

In [20]:
model_file = 'spacycustom-glove-adam_weights_best_20230413_010320.hdf5'

In [21]:

# word_id_spacy + pos_id_default
(fw, tb) = evaluate(model_file,
                    {'main_input_spacy': X_test_spacy,
                     'pos_input_custom': X_test_pos_custom,
                     'dep_input': X_test_dep,
                     'aux_input': X_test_meta}
                    )

print_best_false_true_predicted(fw, tb)

32/32 [==============================] - 2s 12ms/step
Correctly Predicted:  216 / 1267
Accuracy:  0.17048145224940806
*****************************************************************
******************** False statements *************************
*****************************************************************
******************** True Statements *************************
0.3766848
index                                                             274
id                                                          3436.json
label                                                           false
statement           Says his bill, HB 97, would prevent the use of...
subject                                          abortion,health-care
speaker                                                    matt-gaetz
job_title                                        State Representative
state_info                                                    Florida
party                                                

### word_id_custom + pos_id_custom

In [27]:
# word_id_custom + pos_id_custom
train(model_bilstm_customcustom,
      'customcustom-glove-adam',
      {'main_input_custom': X_train_custom, 'pos_input_custom': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_custom': X_val_custom, 'pos_input_custom': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam,
      output_name='main_output_customcustom'
      )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.5910 - categorical_accuracy: 0.2637
Epoch 1: val_categorical_accuracy improved from -inf to 0.26168, saving model to customcustom-glove-adam_weights_best_20230413_010911.hdf5
256/256 [==============================] - 16s 36ms/step - loss: 1.5906 - categorical_accuracy: 0.2646 - val_loss: 1.5866 - val_categorical_accuracy: 0.2617
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.5373 - categorical_accuracy: 0.3139
Epoch 2: val_categorical_accuracy improved from 0.26168 to 0.26869, saving model to customcustom-glove-adam_weights_best_20230413_010911.hdf5
256/256 [==============================] - 7s 27ms/step - loss: 1.5372 - categorical_accuracy: 0.3141 - val_loss: 1.5760 - val_categorical_accuracy: 0.2687
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5232 - categorical_accuracy: 0.3261
Epoch 3: val_categorical_accuracy did not improve from 0.26869
256/256 [==========

#### Evalulation

In [23]:
model_file = 'customcustom-glove-adam_weights_best_20230413_010911.hdf5'

In [24]:
(fw, tb) = evaluate(model_file,
                    {'main_input_custom': X_test_custom,
                     'pos_input_custom': X_test_pos_custom,
                     'dep_input': X_test_dep,
                     'aux_input': X_test_meta}
                    )

print_best_false_true_predicted(fw, tb)

32/32 [==============================] - 2s 14ms/step
Correctly Predicted:  222 / 1267
Accuracy:  0.17521704814522493
*****************************************************************
******************** False statements *************************
*****************************************************************
******************** True Statements *************************
0.44238472
index                                                            1238
id                                                          4250.json
label                                                      pants-fire
statement           Says state Rep. Sandy Pasch, her recall oppone...
subject                                            health-care,unions
speaker                                               alberta-darling
job_title                                 State Senator, 8th District
state_info                                                  Wisconsin
party                                               

## APRIL 13th Evaluation

In [17]:
model_file = 'spacydefault-glove-adam_weights_best_20230413_005106.hdf5'
# word_id_spacy + pos_id_default
(fw, tb) = evaluate(model_file,
                    {'main_input_spacy': X_test_spacy,
                     'pos_input_default': X_test_pos_DEFAULT,
                     'dep_input': X_test_dep,
                     'aux_input': X_test_meta}
                    )

print_best_false_true_predicted(fw, tb)

32/32 [==============================] - 2s 12ms/step
Correctly Predicted:  207 / 1267
Accuracy:  0.1633780584056827
*****************************************************************
******************** False statements *************************
*****************************************************************
******************** True Statements *************************
0.362695
index                                                             602
id                                                          4579.json
label                                                           false
statement           Says Bill Clinton opposes President Barack Oba...
subject                                         message-machine,taxes
speaker                                           american-crossroads
job_title                                                            
state_info                                                           
party                                                  

### APRIL 12th

In [7]:
print("Shape of the embedding matrixes:", embedding_matrix_spacy_100d.shape)
print("Length of the vocabulary dictionary:", vocab_length_spacy)
print("X_train_custom shape:", X_train_spacy.shape)
print("X_val_custom shape:", X_val_spacy.shape)

Shape of the embedding matrixes: (9496, 100)


NameError: name 'vocab_length_spacy' is not defined

In [40]:
# Config 1a
# word_id_custom, pos_custom, dep, meta ADAM optimizer
train(model_bilstm_customcustom,
      'wcustom-poscustom-dep-meta-glove100d-adam',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.5380 - categorical_accuracy: 0.3056
Epoch 1: val_categorical_accuracy improved from -inf to 0.27492, saving model to wcustom-poscustom-dep-meta-glove100d-adam_weights_best.hdf5
256/256 [==============================] - 14s 33ms/step - loss: 1.5380 - categorical_accuracy: 0.3056 - val_loss: 1.5799 - val_categorical_accuracy: 0.2749
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5370 - categorical_accuracy: 0.3067
Epoch 2: val_categorical_accuracy improved from 0.27492 to 0.27570, saving model to wcustom-poscustom-dep-meta-glove100d-adam_weights_best.hdf5
256/256 [==============================] - 7s 28ms/step - loss: 1.5370 - categorical_accuracy: 0.3067 - val_loss: 1.5826 - val_categorical_accuracy: 0.2757
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5351 - categorical_accuracy: 0.3088
Epoch 3: val_categorical_accuracy did not improve from 0.27570
256/256 [======

In [137]:
# Config 1b
##### fixing name exists ######
# word_id_custom, pos_custom, dep, meta SGD optimizer
train(model_bilstm_customcustom,
      'customcustom',
      {'main_input_custom': X_train_custom, 'pos_input_custom': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_custom': X_val_custom, 'pos_input_custom': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=sgd,
      output_name='main_output_customcustom'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.5345 - categorical_accuracy: 0.3092
Epoch 1: val_categorical_accuracy improved from -inf to 0.26947, saving model to customcustom_weights_best_20230412_202519.hdf5
256/256 [==============================] - 25s 69ms/step - loss: 1.5345 - categorical_accuracy: 0.3092 - val_loss: 1.5742 - val_categorical_accuracy: 0.2695
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5335 - categorical_accuracy: 0.3104
Epoch 2: val_categorical_accuracy did not improve from 0.26947
256/256 [==============================] - 16s 62ms/step - loss: 1.5335 - categorical_accuracy: 0.3104 - val_loss: 1.5766 - val_categorical_accuracy: 0.2687
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5316 - categorical_accuracy: 0.3102
Epoch 3: val_categorical_accuracy improved from 0.26947 to 0.28349, saving model to customcustom_weights_best_20230412_202519.hdf5
256/256 [==============================]

In [136]:
# Config 1b AGAIANSDIBSAUI0FBISPA
##### fixing name exists ######
# word_id_custom, pos_custom, dep, meta SGD optimizer
train(model_bilstm_customcustom,
      'customcustom',
      {'main_input_custom': X_train_custom, 'pos_input_custom': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_custom': X_val_custom, 'pos_input_custom': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=sgd,
      output_name='main_output_customcustom'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.6215 - categorical_accuracy: 0.2438
Epoch 1: val_categorical_accuracy improved from -inf to 0.22274, saving model to customcustom_weights_best_20230412_191613.hdf5
256/256 [==============================] - 21s 58ms/step - loss: 1.6215 - categorical_accuracy: 0.2438 - val_loss: 1.6160 - val_categorical_accuracy: 0.2227
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5995 - categorical_accuracy: 0.2515
Epoch 2: val_categorical_accuracy improved from 0.22274 to 0.24844, saving model to customcustom_weights_best_20230412_191613.hdf5
256/256 [==============================] - 15s 57ms/step - loss: 1.5995 - categorical_accuracy: 0.2515 - val_loss: 1.6060 - val_categorical_accuracy: 0.2484
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5949 - categorical_accuracy: 0.2536
Epoch 3: val_categorical_accuracy did not improve from 0.24844
256/256 [==============================]

In [38]:
# Config 1b
# word_id_custom, pos_custom, dep, meta SGD optimizer
train(model_bilstm_customcustom,
      'wcustom-poscustom-dep-meta-glove100d',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=sgd
      )

Epoch 1/30
254/256 [============================>.] - ETA: 0s - loss: 1.6505 - categorical_accuracy: 0.2429
Epoch 1: val_categorical_accuracy improved from -inf to 0.23832, saving model to wcustom-poscustom-dep-meta-glove100d_weights_best.hdf5
256/256 [==============================] - 14s 32ms/step - loss: 1.6503 - categorical_accuracy: 0.2430 - val_loss: 1.6206 - val_categorical_accuracy: 0.2383
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.6055 - categorical_accuracy: 0.2504
Epoch 2: val_categorical_accuracy did not improve from 0.23832
256/256 [==============================] - 7s 26ms/step - loss: 1.6055 - categorical_accuracy: 0.2504 - val_loss: 1.6075 - val_categorical_accuracy: 0.2321
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5969 - categorical_accuracy: 0.2546
Epoch 3: val_categorical_accuracy improved from 0.23832 to 0.24533, saving model to wcustom-poscustom-dep-meta-glove100d_weights_best.hdf5
256/256 [================

In [98]:
# Config 2a
# word_id_spacy, pos_custom, dep, meta, ADAM optimizer
train(model_bilstm_spacycustom,
      'wspacy-poscustom-glove100d-adam',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_spacy, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam
      )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6542 - categorical_accuracy: 0.2368
Epoch 1: val_categorical_accuracy improved from -inf to 0.23832, saving model to wspacy-poscustom-glove100d-adam_weights_best.hdf5
256/256 [==============================] - 16s 40ms/step - loss: 1.6539 - categorical_accuracy: 0.2374 - val_loss: 1.6054 - val_categorical_accuracy: 0.2383
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5962 - categorical_accuracy: 0.2520
Epoch 2: val_categorical_accuracy improved from 0.23832 to 0.24455, saving model to wspacy-poscustom-glove100d-adam_weights_best.hdf5
256/256 [==============================] - 9s 35ms/step - loss: 1.5962 - categorical_accuracy: 0.2520 - val_loss: 1.6006 - val_categorical_accuracy: 0.2445
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5877 - categorical_accuracy: 0.2538
Epoch 3: val_categorical_accuracy improved from 0.24455 to 0.24611, saving model to wspacy-poscust

In [133]:
# Config 3AGAIN
##### INCLUDES TIME IN HDF5 file #####
# word_id_spacy, pos_custom, dep, meta, ADAM optimizer

train(model_bilstm_spacydefault,
      'please',
      {'main_input_spacy': X_train_spacy, 'pos_input_default': X_train_pos_DEFAULT, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input_spacy': X_val_spacy, 'pos_input_default': X_val_pos_DEFAULT, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam,
      output_name='main_output_spacydefault'
      )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.5824 - categorical_accuracy: 0.2665
Epoch 1: val_categorical_accuracy improved from -inf to 0.23520, saving model to please_weights_best_20230412_191224.hdf5


ValueError: Unable to create dataset (name already exists)

In [103]:
# Config 3
##### FIXED BY USING CUSTOM CALLBACK CLASS #####
# word_id_spacy, pos_custom, dep, meta, ADAM optimizer

train(model_bilstm_spacydefault,
      'wspacyy_posdefault_glove100d_adam',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
      optimizer=adam
      )

Epoch 1/30
  5/256 [..............................] - ETA: 8s - loss: 1.5008 - categorical_accuracy: 0.3350WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0346s vs `on_train_batch_end` time: 0.2596s). Check your callbacks.


256/256 [==============================] - ETA: 0s - loss: 1.5300 - categorical_accuracy: 0.3173
Epoch 1: val_categorical_accuracy improved from -inf to 0.26791277527809143. Saving model to wspacyy_posdefault_glove100d_adam_weights_best.hdf5.
256/256 [==============================] - 19s 46ms/step - loss: 1.5300 - categorical_accuracy: 0.3173 - val_loss: 1.5861 - val_categorical_accuracy: 0.2679
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5271 - categorical_accuracy: 0.3146
Epoch 2: val_categorical_accuracy improved from 0.26791277527809143 to 0.28582555055618286. Saving model to wspacyy_posdefault_glove100d_adam_weights_best.hdf5.
256/256 [==============================] - 10s 41ms/step - loss: 1.5271 - categorical_accuracy: 0.3146 - val_loss: 1.5731 - val_categorical_accuracy: 0.2858
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.5203 - categorical_accuracy: 0.3181
Epoch 3: val_categorical_accuracy did not improve from 0.285825550

### APRIL 11th

In [ ]:
train(model_bilstm_dep_meta,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

In [38]:
# Train experiment 3 with dep input and meta input
train(model_bilstm_dep_meta,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta}
     )


Epoch 1/30
254/256 [============================>.] - ETA: 0s - loss: 1.6529 - categorical_accuracy: 0.2455
Epoch 1: val_categorical_accuracy improved from -inf to 0.23364, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 10s 23ms/step - loss: 1.6528 - categorical_accuracy: 0.2451 - val_loss: 1.6216 - val_categorical_accuracy: 0.2336
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.6086 - categorical_accuracy: 0.2486
Epoch 2: val_categorical_accuracy did not improve from 0.23364
256/256 [==============================] - 5s 19ms/step - loss: 1.6086 - categorical_accuracy: 0.2486 - val_loss: 1.6134 - val_categorical_accuracy: 0.2329
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.6029 - categorical_accuracy: 0.2462
Epoch 3: val_categorical_accuracy did not improve from 0.23364
256/256 [==============================] - 5s 19ms/step - loss: 1.6029 - categorical_accuracy: 0.2462 - val_loss: 1.6059 - val_cate

In [57]:
# dep_dict + 1
train(model_bilstm_dep_meta,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta}
     )

Epoch 1/30
256/256 [==============================] - ETA: 0s - loss: 1.5777 - categorical_accuracy: 0.2690
Epoch 1: val_categorical_accuracy improved from -inf to 0.23988, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 15s 36ms/step - loss: 1.5777 - categorical_accuracy: 0.2690 - val_loss: 1.5952 - val_categorical_accuracy: 0.2399
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.5764 - categorical_accuracy: 0.2694
Epoch 2: val_categorical_accuracy improved from 0.23988 to 0.25078, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 8s 30ms/step - loss: 1.5765 - categorical_accuracy: 0.2694 - val_loss: 1.5887 - val_categorical_accuracy: 0.2508
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5751 - categorical_accuracy: 0.2731
Epoch 3: val_categorical_accuracy did not improve from 0.25078
256/256 [==============================] - 8s 29ms/step - loss: 1.5752 - categorical_

In [47]:
# Updated shape of model 3:42am
train(model_bilstm_dep_meta,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'dep_input': X_val_dep, 'aux_input': X_val_meta}
     )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6552 - categorical_accuracy: 0.2353
Epoch 1: val_categorical_accuracy improved from -inf to 0.23832, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 14s 32ms/step - loss: 1.6551 - categorical_accuracy: 0.2353 - val_loss: 1.6208 - val_categorical_accuracy: 0.2383
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.6060 - categorical_accuracy: 0.2513
Epoch 2: val_categorical_accuracy did not improve from 0.23832
256/256 [==============================] - 7s 28ms/step - loss: 1.6059 - categorical_accuracy: 0.2517 - val_loss: 1.6102 - val_categorical_accuracy: 0.2313
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.5967 - categorical_accuracy: 0.2555
Epoch 3: val_categorical_accuracy did not improve from 0.23832
256/256 [==============================] - 7s 27ms/step - loss: 1.5968 - categorical_accuracy: 0.2557 - val_loss: 1.6009 - val_cate

### APRIL 9th

num_steps set to 30
pos dict + 1
meta processed by hardcoding missing values

In [52]:
train(model_bilstm_pos,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6540 - categorical_accuracy: 0.2468
Epoch 1: val_categorical_accuracy improved from -inf to 0.22430, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 14s 25ms/step - loss: 1.6539 - categorical_accuracy: 0.2471 - val_loss: 1.6265 - val_categorical_accuracy: 0.2243
Epoch 2/30
254/256 [============================>.] - ETA: 0s - loss: 1.6086 - categorical_accuracy: 0.2495
Epoch 2: val_categorical_accuracy improved from 0.22430 to 0.23364, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 5s 21ms/step - loss: 1.6088 - categorical_accuracy: 0.2488 - val_loss: 1.6140 - val_categorical_accuracy: 0.2336
Epoch 3/30
253/256 [============================>.] - ETA: 0s - loss: 1.6027 - categorical_accuracy: 0.2481
Epoch 3: val_categorical_accuracy did not improve from 0.23364
256/256 [==============================] - 5s 19ms/step - loss: 1.6028 - categorical_

In [53]:
train(model_bilstm_meta,
      'bilstm_meta',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom, 'aux_input': X_train_meta},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom, 'aux_input': X_val_meta}
     )

Epoch 1/30
254/256 [============================>.] - ETA: 0s - loss: 1.5711 - categorical_accuracy: 0.2720
Epoch 1: val_categorical_accuracy improved from -inf to 0.25234, saving model to bilstm_meta_weights_best.hdf5
256/256 [==============================] - 10s 22ms/step - loss: 1.5709 - categorical_accuracy: 0.2715 - val_loss: 1.5923 - val_categorical_accuracy: 0.2523
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.5696 - categorical_accuracy: 0.2771
Epoch 2: val_categorical_accuracy did not improve from 0.25234
256/256 [==============================] - 5s 20ms/step - loss: 1.5696 - categorical_accuracy: 0.2771 - val_loss: 1.5838 - val_categorical_accuracy: 0.2368
Epoch 3/30
254/256 [============================>.] - ETA: 0s - loss: 1.5701 - categorical_accuracy: 0.2691
Epoch 3: val_categorical_accuracy improved from 0.25234 to 0.26168, saving model to bilstm_meta_weights_best.hdf5
256/256 [==============================] - 5s 19ms/step - loss: 1.5702 - ca

In [65]:
train(model_bilstm_pos,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT},
      {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT}
     )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6631 - categorical_accuracy: 0.2427
Epoch 1: val_categorical_accuracy improved from -inf to 0.22508, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 11s 27ms/step - loss: 1.6629 - categorical_accuracy: 0.2428 - val_loss: 1.6282 - val_categorical_accuracy: 0.2251
Epoch 2/30
256/256 [==============================] - ETA: 0s - loss: 1.6120 - categorical_accuracy: 0.2490
Epoch 2: val_categorical_accuracy improved from 0.22508 to 0.22897, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 5s 20ms/step - loss: 1.6120 - categorical_accuracy: 0.2490 - val_loss: 1.6152 - val_categorical_accuracy: 0.2290
Epoch 3/30
256/256 [==============================] - ETA: 0s - loss: 1.6046 - categorical_accuracy: 0.2462
Epoch 3: val_categorical_accuracy did not improve from 0.22897
256/256 [==============================] - 5s 20ms/step - loss: 1.6046 - categorical_

# APRIL 8th EXPERIMENTS

In [96]:
train(model_bilstm_pos,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6486 - categorical_accuracy: 0.2455
Epoch 1: val_categorical_accuracy improved from -inf to 0.22274, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 13s 32ms/step - loss: 1.6485 - categorical_accuracy: 0.2452 - val_loss: 1.6248 - val_categorical_accuracy: 0.2227
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.6100 - categorical_accuracy: 0.2455
Epoch 2: val_categorical_accuracy did not improve from 0.22274
256/256 [==============================] - 7s 29ms/step - loss: 1.6101 - categorical_accuracy: 0.2452 - val_loss: 1.6167 - val_categorical_accuracy: 0.2220
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.6063 - categorical_accuracy: 0.2454
Epoch 3: val_categorical_accuracy did not improve from 0.22274
256/256 [==============================] - 7s 29ms/step - loss: 1.6062 - categorical_accuracy: 0.2455 - val_loss: 1.6155 - val_cate

In [79]:
### TEST 4: remove pos from equation completely
train(model_bilstm_pos,
      'bilstm',
      {'main_input': X_train_custom},
      {'main_input': X_val_custom}
     )

Epoch 1/30


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: in user code:

    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\input_spec.py", line 197, in assert_input_compatibilit

In [78]:
### TEST 3: attempting to fix test 2:
train(model_bilstm_pos,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

Epoch 1/30


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/embedding_14/embedding_lookup' defined at (most recent call last):
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\fatha\anaconda3\envs\fyp\l

In [66]:
## TEST 2: ERROR = size of pos embeddings
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'model/embedding_2/embedding_lookup' defined at (most recent call last):
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\

In [45]:
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      validation_data=(
	      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom},
      )
      )

C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: Data cardinality is ambiguous:
  x sizes: 10240, 10240, 1284, 1284
  y sizes: 10240
Make sure all arrays contain the same number of samples.

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_custom,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 2

In [ ]:
# CONFIG 2
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_custom},
      validation_data=(
	      {'main_input': X_val_spacy, 'pos_input': X_val_pos_custom},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_custom,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 3

In [ ]:
# CONFIG 3
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_DEFAULT},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_DEFAULT},
      )
      )


In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_DEFAULT,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 4

In [ ]:
# CONFIG 4
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT},
      validation_data=(
          {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_DEFAULT,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 5

In [ ]:
# CONFIG 5
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_custom},
          {'aux_input': X_val_meta},
      )
      )

In [17]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_custom,
                    X_test_meta,
                    # X_test_dep,
                    )

print_best_false_true_predicted(fw, tb)

NameError: name 'evaluate' is not defined

#### CONFIG 6

In [ ]:
# CONFIG 6
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_custom},
      {'aux_input': X_train_meta}
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_custom,
                    X_test_meta,
                    # X_test_dep,
                    )

print_best_false_true_predicted(fw, tb)

# CONFIG 7


In [ ]:
# CONFIG 7
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_DEFAULT},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_DEFAULT},
          {'aux_input': X_val_meta},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_DEFAULT,
                    X_test_meta,
                    # X_test_dep,
                    )
print_best_false_true_predicted(fw, tb)

### CONFIG 8

In [ ]:
# CONFIG 8
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT},
          {'aux_input': X_val_meta},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_DEFAULT,
                    X_test_meta,
                    # X_test_dep,
                    )
print_best_false_true_predicted(fw, tb)